In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = "http://temis.nl/airpollution/no2col/no2regioomimonth_v2.php?Region=9&Year=2011&Month=10"

In [ ]:
response = requests.get(url)

In [ ]:
soup = BeautifulSoup(response.text, 'html.parser')

In [ ]:
soup

In [ ]:

ftest = "http://temis.nl/airpollution/no2col/data/omi/2006/02/no2_200602.grd.zip"

In [ ]:
file = "http://temis.nl/airpollution/no2col/data/omi/data_v2/2011/10/no2_200602.asc.gz"

In [ ]:
import wget

In [ ]:
wget.download(ftest)

In [ ]:
!ls -larth

In [ ]:
|